In [19]:
#libraries
import os
import pandas as pd
from datetime import datetime as dt
from datetime import time as t
import numpy as np
from IPython.display import display

In [20]:
import requests
from requests.auth import HTTPBasicAuth

In [21]:
# source
personal_path = os.environ['USERPROFILE']
if personal_path == 'C:\\Users\\dinhhoang.nguyen.CONCENTRIX':
    middle_path = r'OneDrive - Concentrix Corporation\WFM-internal'
else:
    middle_path = r'Concentrix Corporation\Dinh Hoang Nguyen - WFM-internal'
user_credential = os.path.join(os.environ['USERPROFILE'], middle_path)

In [22]:
update_less = False

In [23]:
# IMPORT MASTER ROSTER
masterroster = pd.read_json(os.path.join(user_credential, r'DB\filejson\masterroster.json'))
masterroster['Employee_ID'] = masterroster['Employee_ID'].astype("Int64")

In [24]:
# IMPORT EPS
if update_less == True:
    eps = pd.read_json(os.path.join(user_credential, r'DB\filejson\eps.json'))
else:
    eps = pd.read_json(os.path.join(user_credential, r'DB\filejson\eps_full.json'))

In [25]:
# IMPORT RAMCO
if update_less == True:
    ramco = pd.read_json(os.path.join(user_credential, r'DB\filejson\ramco.json'))
else:
    ramco = pd.read_json(os.path.join(user_credential, r'DB\filejson\ramco_full.json'))

In [26]:
# IMPORT SCHEDULE
if update_less == True:
    schedule = pd.read_json(os.path.join(user_credential, r'DB\filejson\schedule.json'))
else:
    schedule = pd.read_json(os.path.join(user_credential, r'DB\filejson\schedule_full.json'))

In [27]:
#IMPORT CUIC
if update_less == True:
    cuic = pd.read_json(os.path.join(user_credential, r'DB\filejson\cuic.json'))
else:
    cuic = pd.read_json(os.path.join(user_credential, r'DB\filejson\cuic_full.json'))

In [28]:
#IMPORT IEX ID
iex = pd.read_json(os.path.join(user_credential, r'DB\filejson\iex.json'))

In [29]:
# IMPORT TL
tl = pd.read_json(os.path.join(user_credential, r'DB\filejson\tl.json'))

In [30]:
# IMPORT TRAINING
training = pd.read_json(os.path.join(user_credential, r'DB\filejson\training.json'))

In [31]:
# IMPORT OT
if update_less == True:
    ot = pd.read_json(os.path.join(user_credential, r'DB\filejson\ot.json'))
else:
    ot = pd.read_json(os.path.join(user_credential, r'DB\filejson\ot_full.json'))

In [32]:
# IMPORT REQUIREMENT
requirement = pd.read_json(os.path.join(user_credential, r'DB\filejson\requirement.json'))

In [33]:
# IMPORT WORKPLAN
workplan_merge_iex = pd.read_json(os.path.join(user_credential, r'DB\filejson\workplan_merge_iex.json'))

In [34]:
#IMPORT LABELS
labels = pd.read_json(os.path.join(user_credential, r'DB\filejson\labels.json'))

In [35]:
# BOOKING ATTENDANCE
bkn_att = schedule.drop(columns=['Shift_type'])
bkn_att_ramco = ramco[['EID', 'Date', 'Value']]
bkn_att = pd.merge(bkn_att, tl, on='Emp ID', how='left')
bkn_att = pd.merge(bkn_att, bkn_att_ramco, left_on =['Emp ID', 'Date'], right_on=['EID', 'Date'], how='left')
bkn_att = bkn_att.drop(columns=['EID'])
bkn_att = bkn_att.rename(columns={'Value': 'Ramco'})
#get cuic
cuic['AgentLoggedOnTime'] = cuic['AgentLoggedOnTime'].astype('float')
bkn_att_cuic = cuic[['Employee_ID', 'Session Date','AgentLoggedOnTime']].groupby(['Employee_ID', 'Session Date'], as_index=False).sum()
bkn_att_cuic['Session Date'] = pd.to_datetime(bkn_att_cuic['Session Date'], format='mixed')
bkn_att = pd.merge(bkn_att, bkn_att_cuic, left_on=['Emp ID', 'Date'], right_on=['Employee_ID', 'Session Date'], how='left')
bkn_att = bkn_att.rename(columns={'AgentLoggedOnTime': 'CUIC Actual hrs'})
bkn_att['CUIC Actual hrs'] = bkn_att['CUIC Actual hrs']*24
#remove Employee_ID no CUIC data
bkn_att = bkn_att.drop(columns=['Employee_ID', 'Session Date'])
bkn_att = pd.merge(bkn_att, labels, left_on="Shift", right_on="Attendance Code", how="left")

#sum OT by agent&ID
bkn_att = pd.merge(bkn_att, ot, on=['Emp ID', 'Date'], how="left")
bkn_att['OT'] = bkn_att['OT'].fillna(0)
bkn_att['CUIC Actual hrs'] = bkn_att['CUIC Actual hrs'].fillna(0)
def schedulehrs(x):
    if x['Ramco'] == 'PO' and x['OT'] > 0:
        return x['OT']
    else:
        return x['OT'] + x['Schedule']
bkn_att['Scheduled hrs'] = bkn_att.apply(schedulehrs, axis=1)
bkn_att = bkn_att.rename(columns={'Planned leaves': 'Planned Leave', 'Actual leaves': 'Actual Leave'})
bkn_att['Total scheduled'] = bkn_att['Scheduled hrs'] + bkn_att['Planned Leave']

#Add Attendance

def attendance_func(x):
    if x['Shift'] == 'OFF':
        if x['Ramco'] =='PO':
            result1 = x['CUIC Actual hrs']
        else:
            result1 = 0
    else:
        result1 = x['CUIC Actual hrs']

    if result1 > x['Scheduled hrs']:
        return x['Scheduled hrs']
    else:
        return result1

bkn_att['Attendance'] = bkn_att.apply(attendance_func, axis=1)

#Drop columns
bkn_att = bkn_att.drop(columns=['Description', 'Type', 'filename'])
bkn_att['Date'] = pd.to_datetime(bkn_att['Date'], format ='mixed').dt.strftime('%Y-%m-%d')

In [36]:
# EXPORT TO JSON
if update_less is True:
    bkn_att.to_json(os.path.join(user_credential, r'DB\filejson\booking_attendance.json'), orient='records')
else:
    bkn_att.to_json(os.path.join(user_credential, r'DB\filejson\booking_attendance_full.json'), orient='records')